In [61]:
import networkx as nx
import numpy as np
import torch
from torch_geometric.data import Data
import hicstraw
import pandas as pd
from torch_geometric.utils import convert
import pickle


Different approaches that could be considered to improve the 3D graph:
- find a way to automatically handle outdated gene symbols corresponding to multiple modern gene symbols, so that they could be included in this dataset
- increase flanking region to get more relevant contacts between genes
- consider a different way to define contact strength between two genes: currently this method tries to avoid bin length bias in genes - it only picks one contact bin with highest count to define contact signal between genes
- ignore contact frequency bin below a certain threshold and treat it as zero instead
- consider a different way to define genomic coordinates for genes - currently the coordinates take into account one transcript, not its alternative spliced versions
- speed up the mapping of genes to contacts

In [ ]:
# parameters that I could extend to go into a config file
flank_length = 0 #to include neighbouring regions around the gene to collect more contact frequency information
res = 5000 # in bp, resolution of the contact map
threshold = 0 # values under this threshold are considered to be 0 in a contact map

In [ ]:
# load the csv with gene names corresponding to genomic coordinates
gene_locations = pd.read_csv("slim_hESC_symbols_mapped.csv")
# remove gene names that correspond to more than one gene in symbol mapping
unique_map_gene_locations = gene_locations[gene_locations["Symbol.Alt"].isnull()]

# convert to 0-based indexing
unique_map_gene_locations["Start.BP"] = unique_map_gene_locations["Start.BP"] - 1
unique_map_gene_locations["End.BP"] = unique_map_gene_locations["End.BP"] - 1
# assign a bin ID to each BP start and end 
unique_map_gene_locations["Start.BP.res5kbp"] = unique_map_gene_locations["Start.BP"].values // res
unique_map_gene_locations["End.BP.res5kbp"] = unique_map_gene_locations["End.BP"].values // res

# download relevant Hi-C data from 4DN Data Portal
# downloaded from https://data.4dnucleome.org/files-processed/4DNFI2WSZPG9/#file-overview
hic = hicstraw.HiCFile("4DNFI2WSZPG9.hic")
# highest res possible in this dataset: 1kbp, but we use 5kbp now

# hic.getResolutions() # check available resolutions


In [80]:
contacts = np.array([[1, 2], [0, 4]])
threshold = 2

In [ ]:
np.where(contacts < threshold, 0, contacts)

array([[0, 2],
       [0, 4]])

In [ ]:
n_genes = len(unique_map_gene_locations)

graph_data = []
for i in range(0, n_genes):
    for j in range(i, n_genes):
        gene1 = unique_map_gene_locations.iloc[i]["Symbol"]
        chrname1 = unique_map_gene_locations.iloc[i]["Chromosome"].removeprefix("chr")
        start1 = unique_map_gene_locations.iloc[i]["Start.BP.res5kbp"]
        end1 = unique_map_gene_locations.iloc[i]["End.BP.res5kbp"]

        gene2 = unique_map_gene_locations.iloc[j]["Symbol"]
        chrname2 = unique_map_gene_locations.iloc[j]["Chromosome"].removeprefix("chr")
        start2 = unique_map_gene_locations.iloc[j]["Start.BP.res5kbp"]
        end2 = unique_map_gene_locations.iloc[j]["End.BP.res5kbp"]


        if start1 != 0:
            start1 = start1 - flank_length
        if start2 != 0:
            start2 = start2 - flank_length

        map = hic.getMatrixZoomData(chrname1, chrname2, "oe", "VC_SQRT", "BP", res) 
        contacts = map.getRecordsAsMatrix(start1*res, end1*res, start2*res, end2*res)
        contacts = np.where(contacts < threshold, 0, contacts)
        edge_value = np.max(contacts)
        graph_data.append([gene1, gene2, edge_value])

graph_data = np.array(graph_data)
np.savetxt("3Ddata_graph_hESC_endoderm.csv", graph_data, delimiter=",", fmt="%s")


In [ ]:
graph_data = np.loadtxt("3Ddata_graph_hESC_endoderm.csv", graph_data, delimiter=",")

In [ ]:
# convert the gene contact lists into a NetworkX graph object
G = nx.Graph()

In [58]:
for line in range(0, graph_data.shape[0]):
    G.add_edge(graph_data[line, 0], graph_data[line, 1], value=graph_data[line, 2])


In [ ]:
pagerank = nx.algorithms.link_analysis.pagerank_alg.pagerank(G)
clustering_coef = nx.algorithms.cluster.clustering(G)
betweeness_centrality = nx.betweenness_centrality(G, k=50)

In [62]:

with open("endoderm_nx_graph.pkl", "wb") as f:
    pickle.dump(G, f)